In [1]:
from abc import abstractmethod
from build.Decorators import debug

class Player:
    """
    Parent class for all types of player
    """

    def __init__(self, representationChar) -> None:
        """
        :param representationChar: char to represent the player on the game board
        """
        self.representationChar = representationChar
        self.stats = Stats()

    def __str__(self) -> str:
        """
        __str__ method to output Player object with corresponding char
        :return: representation char
        """
        return str(self.representationChar)

    @abstractmethod
    @debug
    def make_move(self, board):
        """
        Every player has to make his own move
        :param board: current board state
        """
        pass

    @abstractmethod
    @debug
    def give_result(self, result):
        """
        Each player must receive and process feedback
        :param result: Enum element for feedback
        """
        pass

In [ ]:
class Stats:
    """
    An object tracking the stats of a player
    """

    def __init__(self):
        self._won = 0
        self._lost = 0
        self._draw = 0

    def display(self) -> None:

        games = self.played_games()

        print(f"Games     won    |    lost    |    draw    | total games")
        print(f"      {self.get_won():^10} | {self.get_lost():^10} | {self.get_draw():^10} | {games:^10}")
        print(f"      {self.get_won() / games:^10.0%} "
              f"| {self.get_lost() / games:^10.0%} "
              f"| {self.get_draw() / games:^10.0%} "
              f"| {games / games:^10.0%}")

    def incr_won(self) -> None:
        self._won += 1

    def incr_lost(self) -> None:
        self._lost += 1

    def incr_draw(self) -> None:
        self._draw += 1

    def get_won(self) -> int:
        return self._won

    def get_lost(self) -> int:
        return self._lost

    def get_draw(self) -> int:
        return self._draw

    def played_games(self) -> int:
        return self.get_won() + self.get_lost() + self.get_draw()